# Neural Network utilizing ASNM_TUN dataset to determine malicious network traffic

### Importing libraries

In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
np.random.seed(7)
import tensorflow as tf
tf.random.set_seed(2) 

Using TensorFlow backend.


### Pulling Data

In [2]:
df = pd.read_csv("ASNM-datasets/ASNM-TUN.csv")

### Replacing missing spaces with mean of dataframe

In [3]:
df = df.fillna(df.mean())

### Using attributes from Feature Selection for our input data and make out output data label 3
### Label 3: distinguishes among legitimate traffic (symbol 3), direct and obfuscated network attacks (symbols 1 and 2)

In [4]:
FFSCols = ['PolyInd10ordOut[3]',
 'OutPktLen64s10i[8]',
 'OutPkt4s10i[7]',
 'ConTcpFinCntIn',
 'GaussProds4In[1]',
 'FourGonAngleAllN[2]',
 'MedTCPHdrLen',
 'GaussProds8In[5]',
 'SumTTLOut',
 'PolyTime10ordOut[2]',
 'InPkt64s20iTr2KB[14]',
 'OutPktLen64s10i[5]',
 'PolyInd13ordIn[7]',
 'InPkt1s10i[8]',
 'OutPkt32s20iTr4KB[11]',
 'PolyTime10ordOut[8]',
 'OutPktLen4s10i[3]',
 'PolyInd13ordOut[13]',
 'PolyInd13ordIn[12]',
 'InPkt64s20iTr2KB[7]']
X = df[FFSCols].values
y = df[['label_3']].values

### Normalizing input data

In [5]:
scaler = MinMaxScaler(feature_range=(0.1,0.9))
X = scaler.fit_transform(X)


### Encoding the output for multiclassification

In [6]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
y = np_utils.to_categorical(encoded_Y)

### KFold Cross Validation

In [7]:
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

### Building baseline model with Adam optimizer

In [8]:
cvscores = []
for train, test in kfold.split(X, y):
        wandb.init(project="asnm-tun")
        model = Sequential()
        model.add(Dense(X.shape[1]+1, input_dim=X.shape[1], activation='relu'))
        model.add(Dense(40,activation='relu'))
        model.add(Dense(60,activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(10,activation='relu'))
        model.add(Dense(y.shape[1], activation='softmax'))
        opt = keras.optimizers.Adam(learning_rate=0.009)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(
                X[train],
                y[train],
                epochs=1200,
                verbose=0,
                batch_size=10,
                callbacks=[WandbCallback()]
        )
        model.save("model.h5")
        open("model.json", "w").write(model.to_json())
        scores = model.evaluate(X[test], y[test], verbose=0)
        cvscores.append(scores[1] * 100)
        wandb.save('model.h5')
        wandb.save('model.json')
        
        print("---------------------------")
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

---------------------------
accuracy: 87.34%
---------------------------


---------------------------
accuracy: 78.48%
---------------------------


---------------------------
accuracy: 70.89%
---------------------------


---------------------------
accuracy: 87.34%
---------------------------


---------------------------
accuracy: 84.62%
---------------------------
81.73% (+/- 6.32%)


### Building baseline model with SGD optimizer

In [9]:
cvscores = []
for train, test in kfold.split(X, y):
        wandb.init(project="asnm-tun")
        model = Sequential()
        model.add(Dense(X.shape[1]+1, input_dim=X.shape[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(40,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(60,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(30,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(10,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(y.shape[1], activation='softmax'))
        opt = SGD(lr=0.01, momentum=0.75)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.fit(
                X[train],
                y[train],
                epochs=1000,
                verbose=0,
                batch_size=10,
                callbacks=[WandbCallback()]
        )
        model.save(os.path.join(wandb.run.dir, "model.h5")) 
        open(wandb.run.dir + "/model.json", "w").write(model.to_json())
        scores = model.evaluate(X[test], y[test], verbose=0)
        cvscores.append(scores[1] * 100)

        print("---------------------------")
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

---------------------------
accuracy: 75.95%
---------------------------


---------------------------
accuracy: 65.82%
---------------------------


---------------------------
accuracy: 60.76%
---------------------------


---------------------------
accuracy: 63.29%
---------------------------


---------------------------
accuracy: 76.92%
---------------------------
68.55% (+/- 6.64%)


### Building baseline model with Decision Tree

In [10]:
cvscores = []
for train, test in kfold.split(X, y):
    dtree_model = DecisionTreeClassifier(max_depth = 1000).fit(X[train], y[train]) 
    scores = dtree_model.score(X[test], y[test])
    cvscores.append(scores * 100)
    print("---------------------------")
    print("accuracy: %.2f%%" % (scores*100))
    print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

---------------------------
accuracy: 92.41%
---------------------------
---------------------------
accuracy: 83.54%
---------------------------
---------------------------
accuracy: 83.54%
---------------------------
---------------------------
accuracy: 89.87%
---------------------------
---------------------------
accuracy: 84.62%
---------------------------
86.80% (+/- 3.66%)


### Building baseline model with KNN

In [11]:
cvscores = []
for train, test in kfold.split(X, y):
    knn = KNeighborsClassifier(n_neighbors = 3).fit(X[train], y[train]) 
    scores = knn.score(X[test], y[test]) 
    cvscores.append(scores * 100)
    print("---------------------------")
    print("accuracy: %.2f%%" % (scores*100))
    print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

---------------------------
accuracy: 91.14%
---------------------------
---------------------------
accuracy: 78.48%
---------------------------
---------------------------
accuracy: 75.95%
---------------------------
---------------------------
accuracy: 81.01%
---------------------------
---------------------------
accuracy: 84.62%
---------------------------
82.24% (+/- 5.29%)


### Model baseline with Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
cvscores = []
for train, test in kfold.split(X, y):
    rfc = RandomForestClassifier(max_depth=1000).fit(X[train], y[train])  
    scores = rfc.score(X[test], y[test]) 
    cvscores.append(scores * 100)
    print("---------------------------")
    print("accuracy: %.2f%%" % (scores*100))
    print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


---------------------------
accuracy: 93.67%
---------------------------
---------------------------
accuracy: 82.28%
---------------------------
---------------------------
accuracy: 74.68%
---------------------------
---------------------------
accuracy: 86.08%
---------------------------
---------------------------
accuracy: 85.90%
---------------------------
84.52% (+/- 6.16%)


### Building baseline model with SVC

In [13]:
cvscores = []
for train, test in kfold.split(X, y):
    svc_model = OneVsRestClassifier(SVC(kernel = 'linear')).fit(X[train], y[train]) 
    scores = svc_model.score(X[test], y[test])
    cvscores.append(scores * 100)
    print("---------------------------")
    print("accuracy: %.2f%%" % (scores*100))
    print("---------------------------")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


---------------------------
accuracy: 17.72%
---------------------------
---------------------------
accuracy: 5.06%
---------------------------
---------------------------
accuracy: 13.92%
---------------------------
---------------------------
accuracy: 18.99%
---------------------------
---------------------------
accuracy: 19.23%
---------------------------
14.99% (+/- 5.31%)
